In [1]:
import tensorflow as tf
from tensorflow.keras.applications.efficientnet import EfficientNetB7
from tensorflow.keras.models import Model
import os

# Build model
base_model = EfficientNetB7(weights="imagenet")
feature_extractor = Model(
    inputs=base_model.input,
    outputs=base_model.layers[-2].output  # 2560-d features
)

print("Output shape:", feature_extractor.output_shape)

# Save locally
os.makedirs("models", exist_ok=True)
h5_path = "models/efficientnetb7_feature_extractor.h5"
feature_extractor.save(h5_path)

print(f"✅ Saved to {h5_path}")


2026-01-07 13:31:48.140746: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-07 13:31:55.026069: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2026-01-07 13:31:58.998770: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2026-01-07 13:31:59.035065: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2026-01-07 13:32:04.282535: I tensorflow/core/platform/cpu_feature_gua

268326632/268326632 [==============================] - 7s 0us/step
Output shape: (None, 2560)


/home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/tf_keras/src/engine/training.py:3098: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native TF-Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


✅ Saved to models/efficientnetb7_feature_extractor.h5


In [2]:
# =========================
# IMPORTS
# =========================

import boto3
import os
from boto3.s3.transfer import TransferConfig

print("🚀 Starting LOCAL → RunPod S2 transfer (EfficientNet model)")

# =========================
# LOCAL FILE (ALREADY DOWNLOADED / SAVED)
# =========================

# Path to the model you saved locally in SageMaker
LOCAL_FILE_PATH = "models/efficientnetb7_feature_extractor.h5"

if not os.path.exists(LOCAL_FILE_PATH):
    raise FileNotFoundError(f"❌ File not found: {LOCAL_FILE_PATH}")

file_size_mb = os.path.getsize(LOCAL_FILE_PATH) / (1024 ** 2)
print(f"📦 Local model found — {file_size_mb:.2f} MB")

# =========================
# DESTINATION: RUNPOD S2 (EU-RO-1)
# =========================

DEST_BUCKET = "e9tcw5eupu"

# Destination path in RunPod S2
DEST_KEY = "efficientnet/efficientnetb7_feature_extractor.h5"

print("🔐 Creating RunPod S2 client (EU-RO-1)")

runpod_s3 = boto3.client(
    "s3",
    aws_access_key_id="user_37sKcYrvnk9UXaIY3B3Zr90MH0g",
    aws_secret_access_key="rps_YW72UMRXEMRVC8A407OCL08J8G34U1B3QTNO1ETX18pa1n",
    endpoint_url="https://s3api-eu-ro-1.runpod.io",
    region_name="eu-ro-1",
)

print("✅ RunPod S2 client created")

# =========================
# MULTIPART CONFIG
# =========================

transfer_config = TransferConfig(
    multipart_threshold=50 * 1024 * 1024,   # 50 MB
    multipart_chunksize=50 * 1024 * 1024,    # 50 MB
    max_concurrency=2,
    use_threads=True,
)

# =========================
# UPLOAD
# =========================

print("⬆️ Uploading model to RunPod S2")
print(f"Destination: s3://{DEST_BUCKET}/{DEST_KEY}")

runpod_s3.upload_file(
    Filename=LOCAL_FILE_PATH,
    Bucket=DEST_BUCKET,
    Key=DEST_KEY,
    Config=transfer_config,
)

print("🎉 Upload completed successfully!")
print(f"📍 s3://{DEST_BUCKET}/{DEST_KEY}")


🚀 Starting LOCAL → RunPod S2 transfer (EfficientNet model)
📦 Local model found — 246.63 MB
🔐 Creating RunPod S2 client (EU-RO-1)
✅ RunPod S2 client created
⬆️ Uploading model to RunPod S2
Destination: s3://e9tcw5eupu/efficientnet/efficientnetb7_feature_extractor.h5
🎉 Upload completed successfully!
📍 s3://e9tcw5eupu/efficientnet/efficientnetb7_feature_extractor.h5
